In [1]:
import digitalhub as dh

project = dh.get_or_create_project("test-taxonomy")

## Prepare base artifacts

In [12]:
import pandas as pd 

project.log_dataitem('taxonomy_nomi_di_comuni', kind="table", data=pd.read_csv("data-P1/nomi_di_comuni.txt", names=["name"]))
project.log_dataitem('taxonomy_stopwords', kind="table", data=pd.read_csv("data-P1/stopwords.txt", names=["word"]))
project.log_dataitem('taxonomy_termini_specifici', kind="table", data=pd.read_csv("data-P1/termini_specifici.txt", names=["termine"]))

{'kind': 'table', 'metadata': {'project': 'test-taxonomy', 'name': 'taxonomy_termini_specifici', 'version': '1f68fd3bd03b4e9baeec2f7498b7c0b2', 'created': '2025-09-03T13:39:40.517Z', 'updated': '2025-09-03T13:39:40.679Z', 'created_by': 'raman@fbk.eu', 'updated_by': 'raman@fbk.eu', 'embedded': False}, 'spec': {'path': 's3://datalake/test-taxonomy/dataitem/taxonomy_termini_specifici/1f68fd3bd03b4e9baeec2f7498b7c0b2/taxonomy-termini-specifici.parquet', 'schema': {'fields': [{'name': 'termine', 'type': 'string'}]}}, 'status': {'state': 'READY', 'files': [{'path': 'taxonomy-termini-specifici.parquet', 'name': 'taxonomy-termini-specifici.parquet', 'content_type': 'binary/octet-stream', 'size': 1984, 'hash': 'md5:6c8c5277eaefab1a1523077d92f5c05d', 'last_modified': '2025-09-03T13:39:40+00:00'}], 'preview': {'cols': [{'name': 'termine', 'value': ['asilo', 'attività', 'attivitа', 'autostima', 'allattamento', 'azione', 'aiesec', 'alcolismo', 'biotopo', 'barchi']}], 'rows': 82}}, 'user': 'raman@fb

In [5]:
import pandas as pd

tassonomia_df = pd.read_csv('data-P1/tassonomia_comuni.csv', sep=';')
project.log_dataitem(name="tassonomia_comuni", kind="table", data=tassonomia_df)

{'kind': 'table', 'metadata': {'project': 'test-taxonomy', 'name': 'tassonomia_comuni', 'version': '024862365af4482fbeb96ecca6c2c75c', 'created': '2025-09-03T13:25:58.264Z', 'updated': '2025-09-03T13:25:58.362Z', 'created_by': 'raman@fbk.eu', 'updated_by': 'raman@fbk.eu', 'embedded': False}, 'spec': {'path': 's3://datalake/test-taxonomy/dataitem/tassonomia_comuni/024862365af4482fbeb96ecca6c2c75c/tassonomia-comuni.parquet', 'schema': {'fields': [{'name': 'ID_tassonomia', 'type': 'integer'}, {'name': 'azione', 'type': 'string'}, {'name': 'codice_macro', 'type': 'string'}, {'name': 'descrizione_macroambito', 'type': 'string'}, {'name': 'codice_campo', 'type': 'integer'}, {'name': 'descrizione_codice_campo', 'type': 'string'}, {'name': 'codice', 'type': 'string'}]}}, 'status': {'state': 'READY', 'files': [{'path': 'tassonomia-comuni.parquet', 'name': 'tassonomia-comuni.parquet', 'content_type': 'binary/octet-stream', 'size': 10318, 'hash': 'md5:5e87f7a797119e97f0541128db963739', 'last_modi

In [6]:
tassonomia_df = pd.read_csv('data-P1/tassonomia_aziende.csv', sep=';')
project.log_dataitem(name="tassonomia_aziende", kind="table", data=tassonomia_df)

{'kind': 'table', 'metadata': {'project': 'test-taxonomy', 'name': 'tassonomia_aziende', 'version': 'dc83cb965221458a87ebac4375ce7386', 'created': '2025-09-03T13:27:21.097Z', 'updated': '2025-09-03T13:27:21.196Z', 'created_by': 'raman@fbk.eu', 'updated_by': 'raman@fbk.eu', 'embedded': False}, 'spec': {'path': 's3://datalake/test-taxonomy/dataitem/tassonomia_aziende/dc83cb965221458a87ebac4375ce7386/tassonomia-aziende.parquet', 'schema': {'fields': [{'name': 'ID_tassonomia', 'type': 'integer'}, {'name': 'azione', 'type': 'string'}, {'name': 'codice_macro', 'type': 'string'}, {'name': 'descrizione_macroambito', 'type': 'string'}, {'name': 'codice_campo', 'type': 'integer'}, {'name': 'descrizione_codice_campo', 'type': 'string'}, {'name': 'codice', 'type': 'string'}, {'name': "Frequenza di utilizzo dell'azione", 'type': 'integer'}]}}, 'status': {'state': 'READY', 'files': [{'path': 'tassonomia-aziende.parquet', 'name': 'tassonomia-aziende.parquet', 'content_type': 'binary/octet-stream', 's

## Compute azioni comuni

In [26]:
func = project.new_function(name="compute_azioni_comuni",
                            kind="python",
                            python_version="PYTHON3_10",
                            code_src="SuggerimentiComuni.py",
                            handler="taxonomy_suggestions",
                            requirements=["tqdm==4.66.6", "fuzzywuzzy==0.18.0", "python-Levenshtein==0.26.1", "simplemma==1.1.2", "unidecode==1.4.0"]
                           )

In [27]:
run = func.run(action="job",
              inputs={
                  'piani_comunali': project.get_dataitem('piani_comunali').key, 
                  'tassonomia': project.get_dataitem('tassonomia_comuni').key, 
                  'stopwords': project.get_dataitem('taxonomy_stopwords').key, 
                  'nomi': project.get_dataitem('taxonomy_nomi_di_comuni').key, 
                  'termini': project.get_dataitem('taxonomy_termini_specifici').key
              },
              parameters={'soglia': 3, 'soglia_utilizzo': 10},
              wait=True)

2025-09-03 14:01:16,351 - INFO - Waiting for run 4dd07805fc144c24842d5f4cf2f0e95c to finish...
2025-09-03 14:01:21,407 - INFO - Waiting for run 4dd07805fc144c24842d5f4cf2f0e95c to finish...
2025-09-03 14:01:26,430 - INFO - Waiting for run 4dd07805fc144c24842d5f4cf2f0e95c to finish...
2025-09-03 14:01:31,463 - INFO - Waiting for run 4dd07805fc144c24842d5f4cf2f0e95c to finish...
2025-09-03 14:01:36,507 - INFO - Waiting for run 4dd07805fc144c24842d5f4cf2f0e95c to finish...
2025-09-03 14:01:41,531 - INFO - Waiting for run 4dd07805fc144c24842d5f4cf2f0e95c to finish...
2025-09-03 14:01:46,558 - INFO - Waiting for run 4dd07805fc144c24842d5f4cf2f0e95c to finish...
2025-09-03 14:01:51,578 - INFO - Waiting for run 4dd07805fc144c24842d5f4cf2f0e95c to finish...
2025-09-03 14:01:56,600 - INFO - Waiting for run 4dd07805fc144c24842d5f4cf2f0e95c to finish...
2025-09-03 14:02:01,637 - INFO - Waiting for run 4dd07805fc144c24842d5f4cf2f0e95c to finish...
2025-09-03 14:02:06,659 - INFO - Waiting for run 4